In [2]:
import pandas as pd
import numpy as np
import pydit


In [5]:


list_to_check = [ None, np.NaN, pd.NA, pd.NaT, 3,'10',"nan","n/a",'#N/A']

pd.isna(list_to_check)

array([ True,  True,  True,  True, False, False, False, False, False])

In [10]:
l=[1,2,3,"n"]
def is_nan(x):
    return x!=x
for value in l:
    print(f"{repr(value):<8} : {is_nan(value)}")


1        : False
2        : False
3        : False
'n'      : False
